In [3]:
import psycopg2
import random
import requests
import numpy as np
from numpy.random import choice

from bs4 import BeautifulSoup

In [4]:
conn = psycopg2.connect(dbname='admt_g02', user='admt_g02', password='xi6aeSoo', host='alcor.inf.unibz.it', port=5433)
cur = conn.cursor()

In [3]:
cur.execute('Delete from warehouse.operator')
cur.execute('Delete from warehouse.machine')
cur.execute('Delete from warehouse.product')

In [4]:
res = requests.get(f'http://uinames.com/api/?region=austria&amount=100')
operators = res.json()

values = []
for operator in operators:
    values.append(f'(\'{operator["name"]} {operator["surname"]}\')')
    
query_args = ','.join(values)
cur.execute(f'insert into warehouse.operator (operator_name) values {query_args}')

In [5]:
res = requests.get(f'http://uinames.com/api/?amount=250')
machines = [m['surname'] for m in res.json()]

res = requests.get(f'http://uinames.com/api/?amount=20')
vendors = [v['surname'] for v in res.json()]

years = list(range(1995, 2011))

values = []
for machine in machines:
    
    p = np.linspace(20,5,len(vendors))
    p = p / np.sum(p)
    vendor = choice(vendors, p=p)
    
    year = choice(years)   
    values.append(f'(\'{machine}\', \'{vendor}\', {year})')
    
query_args = ','.join(values)
cur.execute(f'insert into warehouse.machine (machine_name, machine_vendor, machine_purchasing_year) values {query_args}')

In [27]:
cur.execute('Select date_id from warehouse.date where day_of_week != 6 and day_of_week != 7 and not holiday order by date_actual')
dates = [d[0] for d in cur.fetchall()]

cur.execute('Select machine_id from warehouse.machine')
machines = [m[0] for m in cur.fetchall()]
p_machines = np.array([random.randint(1,50) for _ in range(len(machines))])
p_machines = p_machines / sum(p_machines)

cur.execute('Select operator_id from warehouse.operator')
operators = [o[0] for o in cur.fetchall()]
p_operators = np.array([random.randint(1,50) for _ in range(len(operators))])
p_operators = p_operators / sum(p_operators)

cur.execute('Select department_name from warehouse.department')
departments = [d[0] for d in cur.fetchall()]

p_department = np.array([random.randint(1,25) for _ in range(len(departments))])
p_department = p_department / sum(p_department)

cur.execute('Select production_stage_id from warehouse.production_stage')
production_stages = [ps[0] for ps in cur.fetchall()]

cur.execute('Select quality_control_id from warehouse.quality_control')
qualtities = [q[0] for q in cur.fetchall()]
p_qualtities = np.array([10,2,3,50,60,300])
p_qualtities = p_qualtities / sum(p_qualtities)

In [8]:
res = requests.get('https://animalcrossingwiki.de/acnl/katalog/einrichtung')
html_soup = BeautifulSoup(res.text, 'html.parser')    

In [9]:
furnitures1 = {}
furnitures2 = {}
furnitures3 = {}
einrichtungssets = html_soup.find("h3", {"id": "einrichtungssets"}).nextSibling.nextSibling.findAll('a', class_= 'wikilink1')


for i, einrichtungsset in enumerate(einrichtungssets):
    title = einrichtungsset['title']
    href = einrichtungsset['href']
    
    department = choice(departments, p=p_department)
    
    res = requests.get(f'https://animalcrossingwiki.de/{href}')
    html_soup_child = BeautifulSoup(res.text, 'html.parser')
    try:
        if(i%3 == 0):
            for furniture in [s.text for s in html_soup_child.find('table').findAll('strong')]:
                furnitures1[furniture] = (title, department)  
        elif(i%3 == 1):
            for furniture in [s.text for s in html_soup_child.find('table').findAll('strong')]:
                furnitures2[furniture] = (title, department)  
        elif(i%3 == 2):
            for furniture in [s.text for s in html_soup_child.find('table').findAll('strong')]:
                furnitures3[furniture] = (title, department)
    except AttributeError:
        pass 

In [29]:
def create_products(furnitures):
    raw_costs = {}
    prod_values = []
    for furniture, values in furnitures.items():
        raw_costs[furniture] = {}
        for i, prod_stage in enumerate(production_stages):
            if(i < 3):
                raw_costs[furniture][prod_stage] = random.randint(10, 500)
            else:
                raw_costs[furniture][prod_stage] = random.randint(0, 50)
        for i in range(1, random.randint(10,200)):
            prod_values.append(f'(\'{furniture}\', \'{furniture+ " - "+ str(i)}\', \'{values[0]}\', \'{values[1]}\')')
    
    query_args = ','.join(prod_values)
    cur.execute(f'Insert into warehouse.product (product_name, product_number, product_category, product_department) values {query_args} returning product_id, product_name')
    return (cur.fetchall(), raw_costs)


def create_production(furnitures, raw_costs, partial_dates):
    prod_values = []
    max_day = len(partial_dates) - 60
    for f_id, furniture in furnitures:
        for prod_stage in production_stages:
            curr_day = random.randint(0,max_day)
            for i, prod_stage in enumerate(production_stages):
                operator = choice(operators, p=p_operators)
                quality = 'NULL'
                if(random.random() < 0.1):
                    quality = choice(qualtities, p=p_qualtities)
                    
                machine = choice(machines, p=p_machines)
                num_d = random.randint(0,20)
                next_day = curr_day + num_d
                duration = num_d * random.randint(1,8) if num_d > 0 else random.randint(1,8)
                
                if(i < 3):
                    raw_cost = random.randint(raw_costs[furniture][prod_stage]-10, raw_costs[furniture][prod_stage]+10)
                else:
                    raw_cost = random.randint(0, raw_costs[furniture][prod_stage]+10)
                
                prod_values.append(f'({operator}, {quality}, {machine}, {prod_stage}, {f_id}, {partial_dates[curr_day]}, \
                {partial_dates[next_day]}, {duration}, {raw_cost})')
                
                curr_day = next_day
                
    query_args = ','.join(prod_values)
    cur.execute(f'Insert into warehouse.production (operator_id, quality_control_id, machine_id, \
    production_stage_id, product_id, start_date_id, end_date_id, duration, raw_material_cost) values {query_args}')
            

In [30]:
cur.execute('Delete from warehouse.product')
products1, raw_costs1 = create_products(furnitures1)
products2, raw_costs2 = create_products(furnitures2)
products3, raw_costs3 = create_products(furnitures3)

In [31]:
create_production(products1, raw_costs1, dates[:int(len(dates)/2)])
create_production(products2, raw_costs2, dates[int(len(dates)/2):])
create_production(products1, raw_costs1, dates)

In [32]:
conn.commit()
cur.close()
conn.close()